# Exercise 3: Dynamic Page Web crawling
Retrieve the average prices of intercity bus connections between the largest k = 5, 10, 50
German cities based on the website
https://global.flixbus.com/.

*Add on:* Visualize this data as a network plot.

### How to get the largest Cities:
Destatis has that data in xlsx format for all german cities:
https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/05-staedte.html

In [2]:
#Load the top n cities to do this:
import pandas as pd
df = pd.read_excel("05-staedte.xlsx",1,header=5).dropna(how="any")

def get_city_name(row:pd.Series)->pd.Series:
    row[14] = row[7].split(",")[0]
    return row
df = df.apply(get_city_name,axis=1)
df = df.rename(columns={1:"nr",2:"land",3:"rb",4:"kreis",5:"verb",6:"gem",7:"stadtbezeichnung",8:"plz",9:"flaeche",10:"einwohner",11:"mann",12:"frau",13:"dichte",14:"stadt"})
n = 5
top_n_df = df.sort_values(by="einwohner",ascending=False).head(n)
top_n_df


,nr,land,rb,kreis,verb,gem,stadtbezeichnung,plz,flaeche,einwohner,mann,frau,dichte,stadt
1,1,11.0,0.0,0.0,0.0,0.0,"Berlin, Stadt",10178.0,891.12,3664088.0,1802038.0,1862050.0,4112.0,Berlin
2,2,2.0,0.0,0.0,0.0,0.0,"Hamburg, Freie und Hansestadt",20038.0,755.09,1852478.0,906933.0,945545.0,2453.0,Hamburg
3,3,9.0,1.0,62.0,0.0,0.0,"München, Landeshauptstadt",80313.0,310.70,1488202.0,725905.0,762297.0,4790.0,München
4,4,5.0,3.0,15.0,0.0,0.0,"Köln, Stadt",50667.0,405.01,1083498.0,528075.0,555423.0,2675.0,Köln
5,5,6.0,4.0,12.0,0.0,0.0,"Frankfurt am Main, Stadt",60311.0,248.31,764104.0,377199.0,386905.0,3077.0,Frankfurt am Main


### How to define the average price between these cities? 
I define the average price as the average price a customer has to pay if he wants to travel within the next two weeks with flixbus as the price seems to differ based on the weekday and on how much in advance a customer books a ticket. Two weeks seems to be enough of a time to cancel out the premium price for late bookings and also seems to cancel out weekday-dependent price differences

Direction needs to be taken into account as the price differs between directions

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

driver = webdriver.Firefox()
driver.get("https://www.flixbus.de/")


In [4]:
#Get rid of cookie modal. selenium had its isssues. this is the only workaround i could find...
webdriver.ActionChains(driver).move_by_offset(475,500).click().perform()

In [5]:
driver.find_element(by=By.XPATH,value="//button[contains(text(),'Verbindung suchen')]").click()

In [20]:
from urllib.parse import unquote
def get_city_ids(list_of_cities:pd.Series) -> dict:
    list_of_cities = list_of_cities.values.tolist()
    dict_of_ids = {}
    for x in range(0,len(list_of_cities),2):
        if x+1 >= len(list_of_cities):
            from_city = list_of_cities[x]
        else:
            from_city, to_city = list_of_cities[x],list_of_cities[x+1]
        from_input, to_input, *other_input = driver.find_elements(by=By.XPATH,value="//input[@class='smhc-input__field']")
        from_input.clear()
        from_input.send_keys(from_city)
        webdriver.ActionChains(driver).move_to_element_with_offset(from_input,30,100).click().perform()
        to_input.clear()
        to_input.send_keys(to_city)
        to_input.send_keys(Keys.ARROW_DOWN)
        webdriver.ActionChains(driver).move_to_element_with_offset(to_input,30,100).click().perform()

        driver.find_element(by=By.XPATH,value="//button[contains(text(),'Verbindung suchen')]").click()
        
        WebDriverWait(driver, 30).until(
            expected_conditions.presence_of_element_located((By.XPATH, "//div[contains(text(),'Hinfahrt')]")) #This is a dummy element
            )

        test_url = driver.current_url.replace("?","&").split("&")
        test_url = [unquote(v) for v in test_url]
        f,t = test_url[3].split("=")[1].split("-")
        #If the URL has some issue sthen try again once:
        if not(f.split("+")[0] in from_city and t.split("+")[0] in to_city):
            print(f.split("+")[0] in from_city, t.split("+"[0] in to_city))
            driver.find_element(by=By.XPATH,value="//button[contains(text(),'Verbindung suchen')]").click()
            WebDriverWait(driver, 30).until(expected_conditions.presence_of_element_located((By.XPATH, "//div[contains(text(),'Hinfahrt')]")))
            
            test_url = driver.current_url.replace("?","&").split("&")
            test_url = [unquote(v) for v in test_url]

        dict_of_ids[from_city] = test_url[1].split("=")[1]
        dict_of_ids[to_city] = test_url[2].split("=")[1]
    return dict_of_ids

In [22]:
id_dict = get_city_ids(top_n_df["stadt"])
def attach_id_to_df(row:pd.Series,id_dict)->pd.Series:
    row["flixuid"] = id_dict[row["stadt"]]
    return row
top_n_df = top_n_df.apply(lambda row : attach_id_to_df(row,id_dict=id_dict),axis=1)
top_n_df

,nr,land,rb,kreis,verb,gem,stadtbezeichnung,plz,flaeche,einwohner,mann,frau,dichte,stadt,flixuid
1,1,11.0,0.0,0.0,0.0,0.0,"Berlin, Stadt",10178.0,891.12,3664088.0,1802038.0,1862050.0,4112.0,Berlin,40d8f682-8646-11e6-9066-549f350fcb0c
2,2,2.0,0.0,0.0,0.0,0.0,"Hamburg, Freie und Hansestadt",20038.0,755.09,1852478.0,906933.0,945545.0,2453.0,Hamburg,40d91e53-8646-11e6-9066-549f350fcb0c
3,3,9.0,1.0,62.0,0.0,0.0,"München, Landeshauptstadt",80313.0,310.70,1488202.0,725905.0,762297.0,4790.0,München,40d901a5-8646-11e6-9066-549f350fcb0c
4,4,5.0,3.0,15.0,0.0,0.0,"Köln, Stadt",50667.0,405.01,1083498.0,528075.0,555423.0,2675.0,Köln,40d91025-8646-11e6-9066-549f350fcb0c
5,5,6.0,4.0,12.0,0.0,0.0,"Frankfurt am Main, Stadt",60311.0,248.31,764104.0,377199.0,386905.0,3077.0,Frankfurt am Main,40d90407-8646-11e6-9066-549f350fcb0c


In [ ]:
flix_url = "https://shop.flixbus.de/search"
depart_flix_id = ""
arrive_flix_id = ""
depart_city = f"departureCity={depart_flix_id}"
arrive_city = f"arrivalCity={arrive_flix_id}"
travel_on = ""
ride_date = f"rideDate={travel_on}"
query_url = f"{flix_url}?{depart_city}&{arrive_city}&{ride_date}&adult=1&_locale=de"